<a href="https://colab.research.google.com/github/ymoslem/Adaptive-MT-LLM/blob/main/other/davinci_finetuning_cost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip3 install tiktoken -q

In [10]:
!pip3 freeze | grep tiktoken

tiktoken==0.4.0


In [11]:
import tiktoken

def encode(text):
  enc = tiktoken.encoding_for_model("davinci")
  encoded = enc.encode(text)
  # print(enc.n_vocab)
  return encoded

encode("This is a test!")

[1212, 318, 257, 1332, 0]

In [12]:
import functools
from concurrent.futures import ThreadPoolExecutor

def encode_batch(batch, num_threads=32):
  encoder = functools.partial(encode)
  with ThreadPoolExecutor(num_threads) as e:
    return list(e.map(encoder, batch))

In [13]:
# test batch encoding
batch = ["Hello!", "this is a test.", "What about this somehow longer sentence?"]
encode_batch(batch)

[[15496, 0],
 [5661, 318, 257, 1332, 13],
 [2061, 546, 428, 7599, 2392, 6827, 30]]

In [14]:
# Count tokens in the source and target datasets

# Change the source and target files
source_file = "tico-19-enes-dedup.en"
target_file = "tico-19-enes-dedup.es"

with open(source_file) as src, open(target_file) as tgt:
  src_sents = src.readlines()
  tgt_sents = tgt.readlines()

  print(f"Number of sentences: {len(src_sents)} + {len(tgt_sents)} = {len(src_sents) + len(tgt_sents)} \n")

  src_sents_encoded = encode_batch(src_sents)
  tgt_sents_encoded = encode_batch(tgt_sents)

  out_src = []
  for src_sent_encoded in src_sents_encoded:
    out_src.extend(src_sent_encoded)
  

  out_tgt = []
  for tgt_sent_encoded in tgt_sents_encoded:
    out_tgt.extend(tgt_sent_encoded)
  
  print(f"Source tokens: {len(out_src)}")
  print(f"Target tokens: {len(out_tgt)}")
  print(f"\nTotal tokens {len(out_src) + len(out_tgt)}")

Number of sentences: 3070 + 3070 = 6140 

Source tokens: 96949
Target tokens: 183786

Total tokens 280735


In [15]:
# Estimate cost of fine-tuning

tokens = len(out_src) + len(out_tgt)
unit = 1000  # tokens
unit_price = 0.03
cost = round((tokens / unit) * unit_price, 2)

print(f"Cost estimation: {cost} USD (excluding VAT)")

Cost estimation: 8.42 USD (excluding VAT)
